In [55]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import ensemble
from sklearn.model_selection import cross_val_score

In [56]:
# Access to data

y2015 = pd.read_csv(
    'D:\\Users\\Borja.gonzalez\\Desktop\\Thinkful-DataScience-Borja\\LoanStats3d.csv',skipinitialspace=True,
    header=1)

D:\Users\Borja.gonzalez\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (0,19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [57]:
#Data cleaning
categorical = y2015.select_dtypes(include=['object'])
for i in categorical:
    column = categorical[i]
    print(i)
    print(column.nunique())

id
421097
term
2
int_rate
110
grade
7
sub_grade
35
emp_title
120812
emp_length
12
home_ownership
4
verification_status
3
issue_d
12
loan_status
7
pymnt_plan
1
url
421095
desc
34
purpose
14
title
27
zip_code
914
addr_state
49
earliest_cr_line
668
revol_util
1211
initial_list_status
2
last_pymnt_d
25
next_pymnt_d
4
last_credit_pull_d
26
application_type
2
verification_status_joint
3


In [58]:
# Drop other columns with many unique variables
y2015.drop(['url', 'emp_title', 'zip_code', 'earliest_cr_line', 'revol_util',
            'sub_grade', 'addr_state', 'desc'], 1, inplace=True)

In [59]:
y2015.tail()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
421092,36271333,38982739.0,13000.0,13000.0,13000.0,60 months,15.99%,316.07,D,5 years,...,0.0,3.0,100.0,50.0,1.0,0.0,51239.0,34178.0,10600.0,33239.0
421093,36490806,39222577.0,12000.0,12000.0,12000.0,60 months,19.99%,317.86,E,1 year,...,1.0,2.0,95.0,66.7,0.0,0.0,96919.0,58418.0,9700.0,69919.0
421094,36271262,38982659.0,20000.0,20000.0,20000.0,36 months,11.99%,664.20,B,10+ years,...,0.0,1.0,100.0,50.0,0.0,1.0,43740.0,33307.0,41700.0,0.0
421095,Total amount funded in policy code 1: 6417608175,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
421096,Total amount funded in policy code 2: 1944088810,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [60]:
# Remove two summary rows at the end that don't actually contain data.
y2015 = y2015[:-2]

In [61]:
# Convert ID and Interest Rate to numeric.
y2015['id'] = pd.to_numeric(y2015['id'], errors='coerce')
y2015['int_rate'] = pd.to_numeric(y2015['int_rate'].str.strip('%'), errors='coerce')

In [62]:
pd.get_dummies(y2015).head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,...,last_credit_pull_d_Nov-2016,last_credit_pull_d_Oct-2015,last_credit_pull_d_Oct-2016,last_credit_pull_d_Sep-2015,last_credit_pull_d_Sep-2016,application_type_INDIVIDUAL,application_type_JOINT,verification_status_joint_Not Verified,verification_status_joint_Source Verified,verification_status_joint_Verified
0,68009401,72868139.0,16000.0,16000.0,16000.0,14.85,379.39,48000.0,33.18,0.0,...,0,0,0,0,0,1,0,0,0,0
1,68354783,73244544.0,9600.0,9600.0,9600.0,7.49,298.58,60000.0,22.44,0.0,...,0,0,0,0,0,1,0,0,0,0
2,68466916,73356753.0,25000.0,25000.0,25000.0,7.49,777.55,109000.0,26.02,0.0,...,0,0,0,0,0,1,0,0,0,0
3,68466961,73356799.0,28000.0,28000.0,28000.0,6.49,858.05,92000.0,21.60,0.0,...,0,0,0,0,0,1,0,0,0,0
4,68495092,73384866.0,8650.0,8650.0,8650.0,19.89,320.99,55000.0,25.49,0.0,...,0,0,0,0,0,1,0,0,0,0


In [63]:
rfc = ensemble.RandomForestClassifier()
X = y2015.drop('loan_status', 1)
Y = y2015['loan_status']
X = pd.get_dummies(X)
X = X.dropna(axis=1)

In [64]:
X.dropna().head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,...,last_credit_pull_d_Nov-2016,last_credit_pull_d_Oct-2015,last_credit_pull_d_Oct-2016,last_credit_pull_d_Sep-2015,last_credit_pull_d_Sep-2016,application_type_INDIVIDUAL,application_type_JOINT,verification_status_joint_Not Verified,verification_status_joint_Source Verified,verification_status_joint_Verified
0,68009401,72868139.0,16000.0,16000.0,16000.0,14.85,379.39,48000.0,33.18,0.0,...,0,0,0,0,0,1,0,0,0,0
1,68354783,73244544.0,9600.0,9600.0,9600.0,7.49,298.58,60000.0,22.44,0.0,...,0,0,0,0,0,1,0,0,0,0
2,68466916,73356753.0,25000.0,25000.0,25000.0,7.49,777.55,109000.0,26.02,0.0,...,0,0,0,0,0,1,0,0,0,0
3,68466961,73356799.0,28000.0,28000.0,28000.0,6.49,858.05,92000.0,21.60,0.0,...,0,0,0,0,0,1,0,0,0,0
4,68495092,73384866.0,8650.0,8650.0,8650.0,19.89,320.99,55000.0,25.49,0.0,...,0,0,0,0,0,1,0,0,0,0


In [65]:
X.isnull().any()

id                                           False
member_id                                    False
loan_amnt                                    False
funded_amnt                                  False
funded_amnt_inv                              False
int_rate                                     False
installment                                  False
annual_inc                                   False
dti                                          False
delinq_2yrs                                  False
inq_last_6mths                               False
open_acc                                     False
pub_rec                                      False
revol_bal                                    False
total_acc                                    False
out_prncp                                    False
out_prncp_inv                                False
total_pymnt                                  False
total_pymnt_inv                              False
total_rec_prncp                

In [66]:
X1 = X.drop(X.columns[[26,88,95,98,131,139,197,198,199,200,201]],axis=1).head()
X1 = X.drop(X[['delinq_2yrs', 'inq_last_6mths','open_acc','pub_rec','revol_bal','total_acc','last_credit_pull_d_Apr-2016',
'last_credit_pull_d_Aug-2015',                  
'last_credit_pull_d_Aug-2016',                  
'last_credit_pull_d_Dec-2014',                 
'last_credit_pull_d_Dec-2015',                  
'last_credit_pull_d_Dec-2016',                  
'last_credit_pull_d_Feb-2015',                  
'last_credit_pull_d_Feb-2016',                  
'last_credit_pull_d_Jan-2015',                  
'last_credit_pull_d_Jan-2016',                  
'last_credit_pull_d_Jan-2017',                  
'last_credit_pull_d_Jul-2015',                 
'last_credit_pull_d_Jul-2016',                 
'last_credit_pull_d_Jun-2015',                
'last_credit_pull_d_Jun-2016',               
'last_credit_pull_d_Mar-2015',              
'last_credit_pull_d_Mar-2016',             
'last_credit_pull_d_May-2015',            
'last_credit_pull_d_May-2016',           
'last_credit_pull_d_Nov-2015',          
'last_credit_pull_d_Nov-2016',         
'last_credit_pull_d_Oct-2015',        
'last_credit_pull_d_Oct-2016',       
'last_credit_pull_d_Sep-2015',      
'last_credit_pull_d_Sep-2016',    
'application_type_INDIVIDUAL',     
'application_type_JOINT',                     
'verification_status_joint_Not Verified',   
'verification_status_joint_Source Verified',  
'verification_status_joint_Verified',
'last_pymnt_d_Apr-2015',
'last_pymnt_d_Apr-2016',            
'last_pymnt_d_Aug-2015',            
'last_pymnt_d_Aug-2016',
'last_pymnt_d_Dec-2015',           
'last_pymnt_d_Dec-2016',           
'last_pymnt_d_Feb-2015',           
'last_pymnt_d_Feb-2016',           
'last_pymnt_d_Jan-2015',           
'last_pymnt_d_Jan-2016',           
'last_pymnt_d_Jan-2017',           
'last_pymnt_d_Jul-2015',           
'last_pymnt_d_Jul-2016',            
'last_pymnt_d_Jun-2015',            
'last_pymnt_d_Jun-2016',            
'last_pymnt_d_Mar-2015',            
'last_pymnt_d_Mar-2016',            
'last_pymnt_d_May-2015',            
'last_pymnt_d_May-2016',            
'last_pymnt_d_Nov-2015',            
'last_pymnt_d_Nov-2016',            
'last_pymnt_d_Oct-2015',            
'last_pymnt_d_Oct-2016',            
'last_pymnt_d_Sep-2015',            
'last_pymnt_d_Sep-2016',           
'next_pymnt_d_Feb-2017',            
'next_pymnt_d_Jan-2017',           
'next_pymnt_d_Jul-2016',            
'next_pymnt_d_Mar-2017',
'last_credit_pull_d_Apr-2015',
'purpose_wedding',                                    
'title_Business',                                     
'title_Car financing',                                
'title_Credit Card/Auto Repair',                      
'title_Credit card refinancing',                      
'title_Debt consolidation',                           
'title_DebtC',                                        
'title_Green loan',                                   
'title_Home buying',                                  
'title_Home improvement',                             
'title_Learning and training',                        
'title_Major purchase',                               
'title_Medical expenses',                             
'title_Moving and relocation',                        
'title_New Baby and New House (CC Consolidate)',      
'title_Other',                                        
'title_Pay off Lowes Card',                           
'title_Paying off higher interest cards & auto',      
'title_Prescription Drug and Medical Costs',          
'title_SAVE',                                         
'title_Simple Loan Until Contract Is Completed',      
'title_Student Loan',                                 
'title_Trying to come back to reality!',              
'title_Vacation',                                     
'title_considerate',                                  
'title_new day',                                      
'title_new kitchen for momma!',                       
'title_odymeds',                                      
'initial_list_status_f',                              
'initial_list_status_w',
'home_ownership_RENT',
'verification_status_Not Verified',       
'verification_status_Source Verified',    
'verification_status_Verified',           
'issue_d_Apr-2015',                       
'issue_d_Aug-2015',                       
'issue_d_Dec-2015',                       
'issue_d_Feb-2015',                       
'issue_d_Jan-2015',                       
'issue_d_Jul-2015',                       
'issue_d_Jun-2015',                       
'issue_d_Mar-2015',                       
'issue_d_May-2015',                       
'issue_d_Nov-2015',                       
'issue_d_Oct-2015',                       
'issue_d_Sep-2015',                       
'pymnt_plan_n',                           
'purpose_car',                            
'purpose_credit_card',                    
'purpose_debt_consolidation',             
'purpose_educational',                    
'purpose_home_improvement',               
'purpose_house',                          
'purpose_major_purchase',                 
'purpose_medical',                        
'purpose_moving',                         
'purpose_other',                          
'purpose_renewable_energy',              
               'purpose_small_business', 
'purpose_vacation', 
'term_ 36 months',                
'term_ 60 months',                
'grade_A',                        
'grade_B',                        
'grade_C',                        
'grade_D',                        
'grade_E',                        
'grade_F',                        
'grade_G',                        
'emp_length_1 year',              
'emp_length_10+ years',           
'emp_length_2 years',             
'emp_length_3 years',             
'emp_length_4 years',             
'emp_length_5 years',             
'emp_length_6 years',             
'emp_length_7 years',             
'emp_length_8 years',             
'emp_length_9 years',             
'emp_length_< 1 year',            
'emp_length_n/a',                 
'home_ownership_ANY',             
'home_ownership_MORTGAGE',        
'home_ownership_OWN',
'num_accts_ever_120_pd',        
'num_actv_bc_tl',                
'num_actv_rev_tl',               
'num_bc_sats',                   
'num_bc_tl',                     
'num_il_tl',                     'num_op_rev_tl',   
'num_rev_tl_bal_gt_0',           
'num_sats',                      
'num_tl_30dpd',                  
'num_tl_90g_dpd_24m',            
'num_tl_op_past_12m',            
'pct_tl_nvr_dlq',                
'pub_rec_bankruptcies',          
'tax_liens',                     
'tot_hi_cred_lim', 'out_prncp'            
              ]],axis=1)
X1.dtypes

id                              int64
member_id                     float64
loan_amnt                     float64
funded_amnt                   float64
funded_amnt_inv               float64
int_rate                      float64
installment                   float64
annual_inc                    float64
dti                           float64
out_prncp_inv                 float64
total_pymnt                   float64
total_pymnt_inv               float64
total_rec_prncp               float64
total_rec_int                 float64
total_rec_late_fee            float64
recoveries                    float64
collection_recovery_fee       float64
last_pymnt_amnt               float64
collections_12_mths_ex_med    float64
policy_code                   float64
acc_now_delinq                float64
tot_coll_amt                  float64
tot_cur_bal                   float64
total_rev_hi_lim              float64
acc_open_past_24mths          float64
avg_cur_bal                   float64
chargeoff_wi

In [ ]:
cross_val_score(rfc, X1, Y, cv=10)

array([ 0.92237183,  0.96829807,  0.96642207,  0.9636437 ,  0.96369033,
        0.96274044,  0.96214586,  0.96145717,  0.96136034,  0.96012445])

In [ ]:
cross_val_score(rfc, X1, Y, cv=10).mean()

Can you do it without using anything related to payment amount or outstanding principal?
It cannot be done without considering the payment amount or outstanding principal due to the correlation between these variables and the explained variable. Once they are taken out, the cross validation score drops to 70%.